In [1]:
#bibliotecas utilizadas
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Database
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


### Parte 1 - Dataset

Ao iniciar qualquer experimento com Inteligência Artificial, o primeiro passo, e um dos mais importantes, é compreender o problema interpretando a base de dados escolhida. Dessa forma, nesta parte deve-se:
* Carregar o dataset
* Analisar os atributos, excluindo atributos irrelevantes.

In [2]:
def readDataset(filename, classIndex):
    """Função de leitura e preprocessamento do dataset, recebe como entrada o nome do 
    arquivo e o indice do atributo alvo. Essa função retorna o dataframe de forma
    que o atributo alvo esteja na última coluna.
    """
    df = pd.read_csv(filename, delimiter=';')
    df_classes = df.iloc[:,classIndex] 
    #deleta colunas, para remover outras colunas, deve-se adicionar seus índices entre os 
    #colchetes, separados por vírgula após a variável classIndex
    df.drop(df.columns[[classIndex]], axis=1, inplace=True)
    df['Hotel Stars'] = df_classes
    return df

### Parte 2 - Calcular as frequências da saída 

No aprendizado bayesiano, probabilidades condicionais são calculadas a partir do atributo alvo.
Dessa forma, a etapa inicial é analisar todas as instâncias e calcular a frequência de cada
classe.

In [3]:
def getOutputFreq(df, index):
    """Função que calcula a frequência das classes de saída, recebe como entrada o dataframe
    completo e o índice da coluna dos atributos alvo.
    Retorna um dicionário com a frequêNcia de cada classe e os respectivos nomes;
    """
    outputNames = df.iloc[:,index].unique()
    totalOutputFreq = df.iloc[:,index].count()
    classesDic = {}
    for i in range(len(outputNames)):
        outputFreq = df.iloc[:,index][df.iloc[:,index] == outputNames[i]].count()
        classesDic[i] = [outputFreq, outputFreq/totalOutputFreq]
    return outputNames, classesDic

### Parte 3 - Treinamento

Calculadas as frequências das classes, o Naive Bayes irá calcular a probabilidade de cada 
atributo. Para isso, deve-se, para cada coluna, calcular a probabilidade condicional desse 
algoritmo dado uma saída X. 


In [4]:
from collections import defaultdict
def training(df, outputNames, classIndex):
    """Recebe como entrada o dataframe completo, o índice do atributo alvo e os nomes
    das classes. Para cada instância, busca o valor de um atributo e calcula a probabilidade
    condicional do mesmo para todas as classes. Esse processo é repetido até que todos os 
    atributos sejam analisados
    Retorna: um dicionário com todas as probabilidades de todos os valores das instâncias;
    """
    dic = defaultdict(dict)
    dicNames = defaultdict(dict)
    for j in range(len(outputNames)): #para todas as classes
        for i in range(df.shape[1]-1): #percorre até que todos os atributos sejam analisados
            attributeNames = df.iloc[:,i].unique() #retorna os valores do atributo atual
            dicNames[j][i] = attributeNames #adiciona o nome do atributo ao dicionário
            prob = np.ones(len(attributeNames))
            for k in range(len(attributeNames)): #para cada valor de atributo, calcular a probabilidade
                prob[k] = df.iloc[:,classIndex][(df.iloc[:,classIndex] == outputNames[j])][df.iloc[:,i]== attributeNames[k]].count()
                if prob[k] == 0:
                    prob[k] == 0.1
            dic[j][i] = prob #adiciona no dicionário
    return dic, dicNames

### Parte 4 - Teste

Com as probabilidades obtidas com a base de treinamento, precisa-se verificar o sucesso do algoritmo
com novas instâncias. Para isso, deve-se primeiramente fazer uma busca no dicionário o valor
de cada atributo da instância analisada, caso esse valor não exista no dicionário, deve-se adicionar
uma probabilidade igual a 0.01, caso esse valor exista, a probabilidade do atributo é retornada 
dividindo-se pela frequência da classe. Esse processo repete-se para cada atributo da instância,
de forma que a probabilidade final é uma multiplicação de todos os atributos dessa instância. 
Ao final, multiplica-se a probabilidade obtida pela probabilidade da classe atual, sendo essa
a probabilidade final da instância. 

O processo é repetido para todas as classes de cada instância até que todas as instâncias e
classes sejam analisadas. Onde a predição final de cada instância será dada pela classe com
maior probabilidade encontrada.

In [5]:
def test(training, outputNames,dic,dicNames,classesDic):
    p = [[]]
    y_pred = []
    for i in range(len(training)): #para cada instância do treinamento
        for k in range(len(outputNames)): #para cada classe 
            prob = 1
            for j in range(len(training[i])-1): #para cada atributo da instância
                index = np.where(dicNames[k][j]==training[i][j]) #encontre o valor do atributo no dicionário
                if np.array(index).size==0: #se não existir, adicione uma probabilidade baixa
                    prob*=0.01
                else: #se existir, recupere a probabilidade do valor encontrado e divida pela frequencia da classe analisada
                    prob*=(dic[k][j][index]/classesDic[k][0]) #multiplique o valor encontrado pelo valor dos demais atributos da instacia
            prob*=classesDic[k][1] #multiplique a probabilidade de todos os atributos pela probabilidade da classe
            p[i].append(prob) #guarde todas as probabilidades encontradas 
        y_pred.append(np.argmax(p[i])) #retorne o índice da classe que obtiver maior probabilidade
        if i < len(training):
            p.append([])
    return y_pred,p
    

In [6]:
def decodeLabels(X_test,outputNames):
    """Função auxiliar para codificar a saída de acordo com os valores das classes encontrados
    no dicionário, considerando que o dicionário pode ordenar as classes com índices diferentes
    dos índices da saída.
    Recebe como entrada a base de treino eos nomes das classes
    Retorna a saída codificada
    """
    y_test = []
    for i in range(X_test.shape[0]):
        for j in range(len(outputNames)):
            if X_test.iloc[i,19] == outputNames[j]:
                y_test.append(j)
    return y_test

### Algoritmo Final

In [7]:
def naiveBayes():
    df = readDataset('LasVegasTripAdvisorReviews-Dataset.csv',14)
    X_train, X_test = train_test_split(df, test_size=0.33,random_state=0)
    outputNames,classesDic = getOutputFreq(X_train,19)
    print(outputNames)
    dic,dicNames = training(X_train, outputNames,19)
    y_pred,p = test(df.values,outputNames,dic,dicNames,classesDic)
    y_test = decodeLabels(df, outputNames)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    return  accuracy_score(y_test, y_pred)
    return y_pred

In [8]:
naiveBayes()

['3' '4' '5' '4,5' '3,5']
             precision    recall  f1-score   support

          0       0.61      1.00      0.76        96
          1       1.00      0.82      0.90       120
          2       1.00      0.93      0.96       192
          3       1.00      0.71      0.83        24
          4       1.00      0.74      0.85        72

avg / total       0.93      0.88      0.89       504

[[ 96   0   0   0   0]
 [ 22  98   0   0   0]
 [ 14   0 178   0   0]
 [  7   0   0  17   0]
 [ 19   0   0   0  53]]


0.87698412698412698